### Тестове завдання
### Задача 2
#### Виконав Михайлов Валерій

## Спочатку завантажемо вже знайомі дані.

In [146]:
import pandas as pd


# Uploading CSV files
directory = 'S_Data'

sellers = pd.read_csv(f'{directory}/sellers.csv')
products = pd.read_csv(f'{directory}/products.csv')
product_category_name_translation = pd.read_csv(f'{directory}/product_category_name_translation.csv')
orders = pd.read_csv(f'{directory}/orders.csv')
order_reviews = pd.read_csv(f'{directory}/order_reviews.csv')
order_payments = pd.read_csv(f'{directory}/order_payments.csv')
order_items = pd.read_csv(f'{directory}/order_items.csv')
geolocation = pd.read_csv(f'{directory}/geolocation.csv')
customers = pd.read_csv(f'{directory}/customers.csv')

# Виконаємо класифікацію текстів коментарім за емоційним забарвленням на 3 категорії.

Використаємо натреновану модель для аналізу сентименту комментаря.

In [147]:
order_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


Нам знадобиться лише заголовок комментаря, його текст та id.

In [148]:
df = order_reviews[['review_id', 'review_comment_title', 'review_comment_message']].set_index('review_id')

df.fillna('', inplace=True)
df['comment'] = df['review_comment_title'] + "\n" + df['review_comment_message']
df = df[df['comment'] != '\n'][['comment']]

## 1. Проведемо розділення комментарів на 3 категорії за допомогою натренованої багатомовної моделі BERT та тої, яка налаштована спеціально під португалську.

In [149]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


# bert
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
    sentiment = torch.argmax(probabilities).item()
    labels = ["NEG", "NEU", "POS"]

    # distibution of emotional score
    neg_prob = probabilities[0] + 2/3 * probabilities[1]
    neu_prob = 1/3 * probabilities[1] + probabilities[2] + 1/3 * probabilities[3]
    pos_prob = 2/3 * probabilities[3] + probabilities[4]

    return labels[torch.argmax(torch.tensor([neg_prob, neu_prob, pos_prob])).item()]


# pt_bert
tokenizer_pt = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model_pt = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased")
def analyze_sentiment_pt(text):
    inputs = tokenizer_pt(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model_pt(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
    sentiment = torch.argmax(probabilities).item()
    labels = ["NEG", "NEU", "POS"]


    return labels[int(torch.argmax(outputs.logits))]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [150]:
df_small = df[:10].copy()
df_small['sentiment'] = df_small['comment'].apply(analyze_sentiment)
df_small['sentiment_pt'] = df_small['comment'].apply(analyze_sentiment_pt)

Також додамо переклад португалських речень на англійську, щоб порівняти перфоманси 
моделей, і обрати, яка більше нам підходить.

In [151]:
from deep_translator import GoogleTranslator


translator = GoogleTranslator()
def translate_text(text):
    return translator.translate(text=text, source='sp', target='en')

In [155]:
df_small['comment_en'] = df_small['comment'].apply(translate_text)

In [156]:
df_small[['comment', 'comment_en', 'sentiment', 'sentiment_pt']].style.hide(axis='index')

comment,comment_en,sentiment,sentiment_pt
Recebi bem antes do prazo estipulado.,I received it well before the stipulated deadline.,NEU,NEG
Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,"Congratulations Lannister Stores, I loved buying online, it's safe and practical. Congratulations to everyone, happy Easter.",POS,NEG
recomendo aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,I recommend an efficient device. On the website the brand of the device is printed as 3desinfector and when it arrives it has another name... update with the correct brand since it is the same device,POS,NEG
"Mas um pouco ,travando...pelo valor ta Boa.","But a little bit, freezing... for the price it's good.",NEU,NEG
"Super recomendo Vendedor confiável, produto ok e entrega antes do prazo.","I highly recommend Reliable seller, ok product and delivery before the deadline.",POS,NEG
"GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI E ESSA COMPRA AGORA ME DECPCIONOU","I WOULD LIKE TO KNOW WHAT HAPPENED, I ALWAYS RECEIVED IT AND THIS PURCHASE NOW DISAPPOINTED ME",NEG,NEG
Não chegou meu produto Péssimo,My product didn't arrive Terrible,NEG,NEG
Ótimo Loja nota 10,Great Store rating 10,POS,NEG
obrigado pela atençao amim dispensada,Thank you for your attention,NEG,NEG
"A compra foi realizada facilmente. A entrega foi efetuada muito antes do prazo dado. O produto já começou a ser usado e até o presente, sem problemas.","The purchase was made easily. The delivery was made well before the deadline. The product has already been used and so far, without any problems.",POS,NEG


Звернемо увагу на коментарі, де оцінки моделей відрізняються і оберемо кращу модель: 
- 3 коментар важко віднести до якоїсь оцінки,
- 2, 5, 6, 7, 8 та 10 коментарі правильно класифікувала перша модель,
- 9 коментар правильно класифікувала друга модель.

Зважаючи на такі результати та більшу універсальність першої моделі, будемо
визначати сентимент саме за допомогою неї.

In [157]:
df['sentiment'] = df['comment'].apply(analyze_sentiment)

Перевіримо модель на 3 неочевидних комментарях:
- "I didn't have to wait long, the product was packaged well, I'm terribly pleased.",
- "Not a bad product, delivery arrived on time",
- "I'm not happy, bad purchase :(".

In [158]:
print("1: ", analyze_sentiment("Não tive de esperar muito tempo, o produto foi muito bem embalado e estou muito satisfeita."))
print("2: ", analyze_sentiment("Não é um mau produto, a entrega chegou a tempo"))
print("3: ", analyze_sentiment("Não estou satisfeito, má compra :("))

1:  POS
2:  NEU
3:  NEG


Модель працює і це чудово!

# Розробимо підхід до пошуку ціни в коментарях.

Можна використати вже готові NER (Named Entity Recognition) моделі, бо 
вони вже натреновані розрізняти окремі сутності в реченнях, тому з виділенням ціни
для них не буде проблем, але я використав більш простий, але також працюючий підхід 
з регулярними виразами.

In [159]:
import re

def find_prices(text):
    number_words = {
        'um': '1', 'dois': '2', 'três': '3', 'quatro': '4', 'cinco': '5',
        'seis': '6', 'sete': '7', 'oito': '8', 'nove': '9', 'dez': '10',
        'vinte': '20', 'trinta': '30', 'quarenta': '40', 'cinquenta': '50',
        'sessenta': '60', 'setenta': '70', 'oitenta': '80', 'noventa': '90',
        'cem': '100', 'duzentos': '200', 'trezentos': '300', 'quatrocentos': '400',
        'quinhentos': '500', 'seiscentos': '600', 'setecentos': '700',
        'oitocentos': '800', 'novecentos': '900', 'mil': '1000'
    }
    pattern = r'''
        (?:R?\$\s*|\$\s*|BRL\s*|USD\s*)\d+(?:[.,]\d+)?  # Prices with currency symbols
        |\d+(?:[.,]\d+)?\s*(?:BRL|USD|R\$|\$)           # Prices with currency designations after the number
        |(\d+(?:[.,]\d+)?)\s*(?:reais?|dólares?)        # Prices with word-based currency designations
        |({})\s*(?:reais?|dólares?)                     # Word numbers with currency designations
    '''.format('|'.join(number_words.keys()))

    matches = re.finditer(pattern, text, re.VERBOSE | re.IGNORECASE)
    prices = []

    for match in matches:
        if match.group(1):  # If a number with a word-based currency designation is found
            prices.append(match.group(0))
        elif match.group(2):  # If a word number is found
            word = match.group(2).lower()
            number = number_words.get(word, word)
            currency = 'reais' if 'real' in match.group(0).lower() else 'dólares'
            prices.append(f"{number} {currency}")
        else:
            prices.append(match.group(0))

    return prices


Спробуємо розрізнити переклад такого речення: "The product costs BRL 50.00 or $10.00 USD. We also have options for 75.50 BRL and 25 reais. A premium version costs two hundred reais.".

In [160]:
# Example usage
text = "O produto custa R$ 50,00 ou $ 10,00 USD. Também temos opções por 75,50 BRL e 25 reais. Uma versão premium custa duzentos reais."
prices = find_prices(text)
for price in prices:
    print(f"Price: {price}")

Price: R$ 50,00
Price: $ 10,00
Price: 75,50 BRL
Price: 25 reais
Price: 200 dólares


Цей підхід надійно працює і його легко покращувати.

# Виконаємо аналіз.
Які саме показники цікавлять зазначено не було, тому я виведу інформацію, яка
є цікавою для мене: найкращі товари та найгірші продавці.

# 1.
Для аналізу товарів я буду використовувати середнє значення оцінки покупцями та кількість
продажів (тобто найпопулярніші товари з високими оцінками).

In [161]:
# product_id & review score

data = pd.merge(orders[orders['order_approved_at'].notna()], order_items, on='order_id', how='left')
data = pd.merge(data, products, on='product_id')
data = pd.merge(data, order_reviews, on='order_id')
data = pd.merge(data, product_category_name_translation, on='product_category_name')

df_product = data[['product_id', 'review_score']]

df_product = df_product.groupby('product_id').agg(
    avg_score=('review_score', 'mean'),
    count=('review_score', 'count')
).reset_index().sort_values(['avg_score', 'count'], ascending=False)

df_product = pd.merge(df_product, products, on='product_id')
df_product = pd.merge(df_product, product_category_name_translation, on='product_category_name')


Виведемо найпопулярніші товари з високими оцінками.

In [162]:
df_product[['product_id', 'product_category_name_english', 'avg_score', 'count']].head(10).style.hide(axis='index')

product_id,product_category_name_english,avg_score,count
37eb69aca8718e843d897aa7b82f462d,garden_tools,5.000000,15
03e1c946c0ddfc58724ff262aef08dff,housewares,5.000000,14
8173ca41cdb176462f9ae79821a48404,sports_leisure,5.000000,14
870bcc6c58e03ca658cfdd13db4bbe28,computers_accessories,5.000000,14
8b6ed41158b194711f83b8da92757544,sports_leisure,5.000000,14
2722b7e5f68e776d18fe901638034e54,health_beauty,5.000000,13
143c616ec3fe0cb065c3805cf9bd812e,garden_tools,5.000000,12
16ce899c7af0c99f46948734a0d00f0f,stationery,5.000000,12
2ced45a3059cbb128a8b8b6508809746,computers_accessories,5.000000,11
4c8b28305f570899b6ded964ddd234a9,furniture_decor,5.000000,11


## 2.
Тепер знайдемо найгіршого продавця, але вже будемо орієнтуватися на середню семантичну 
оцінку коментаря, бо вона відорбражає не тільки якість товару, але й комунікацію з людьми.

In [163]:
df = pd.merge(df, order_reviews, on='review_id')
df = pd.merge(df, orders, on='order_id')
df = pd.merge(df, order_items, on='order_id')


emo_to_score = category_mapping = {
    'NEG': 1,
    'NEU': 2,
    'POS': 3
}
df['sentiment_score'] = df['sentiment'].map(emo_to_score)

In [164]:
df_sorted = df.groupby('seller_id').agg(
    avg_sentiment=('sentiment_score', 'mean'),
    count=('sentiment_score', 'count')
).reset_index().sort_values(['count', 'avg_sentiment'], ascending=[False, True])

In [165]:
df_sorted[['seller_id', 'avg_sentiment', 'count']].head(10).style.hide(axis='index')

seller_id,avg_sentiment,count
4a3ca9315b744ce9f8e9374361493884,2.062162,1110
6560211a19b47992c3666cc44a7e94c0,2.091346,1040
1f50f920176fa81dab994f9023523100,2.116601,1012
cc419e0650a3c5ba77189a1882b7556a,2.174603,819
7c67e1448b00f6e969d365cea6b010ab,1.742240,741
1025f0e2d44d7041d6cf58b6550e0bfa,1.906425,716
da8622b14eb17ae2831f4ac5b9dab84a,2.132460,687
955fee9216a65b617aa5c0531780ce60,2.116923,650
ea8482cd71df3c1969d7b9473ff13abc,2.074468,564
7a67c85e85bb2ce8582c35f2203ad736,2.379648,511


Найгірший продавець серед популярних знайдений!